# modisLandsat - notebook to calibrate and upload hierarchical random forests for Landsat LAI and fAPAR based on MODIS algorithms

## richard.fernandes@canada.ca

In [20]:
# Modules

In [21]:
# python modules 
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_text
pd.options.mode.chained_assignment = None  # default='warn'
import ee
import geemap 

In [22]:
# construct hierarchal random forests for FTL method
def hierarchicalRF(dataDictParent,dataDictChild,regressorsNames,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=20, maxDepthChild=20, minSamplesSplit=11,maxleafnodesParent= 100, minSamplesLeafParent=10, maxleafnodesChild= 999, minSamplesLeafChild=10,maxFeatures="auto",nTrees = 100):
    
    
    # make generic names for regressors for use in GEE
    regressors = []
    for item in np.arange(1,len(regressorsNames)+1,1):
        regressors.append('x' + str(item))

    # Calibrate hierarchal randforest preeidctors for each biome
    for biome in dataDictParent.keys() : 
        print('biome:',biome)

        # subset only the regressors and response 
        dfBiome = dataDictChild[biome]['DF'][sum([regressorsNames,response],[])].astype('int')   
        dfParent = dataDictParent[biome]['DF'][sum([regressorsNames,response],[])].astype('int')


        dfBiome.columns = sum([regressors,response],[])
        dfParent.columns = sum([regressors,response],[])
        print('Total size ',dfBiome.shape)
        print('Parent size ',dfParent.shape)
         # populate a parent RF dictionary that holds a single tree RF used to partition data into child RFs
        parentRFDict = {}
        parentRFDict.update({'regressors': regressors})
        parentRFDict.update({'regressorsGEE': regressorsGEENames})
        parentRFDict.update({'response': response})
        parentRFDict.update({'domain':makeDomain(dfBiome[regressors],domainIndex,domainScaling,domainOffset)})
        parentRFDict.update({'RF': RandomForestRegressor(n_estimators=1,min_samples_leaf=minSamplesLeafParent,min_samples_split=minSamplesSplit,bootstrap=False,random_state=0,verbose=0,max_depth=maxDepthParent,max_leaf_nodes=maxleafnodesParent,max_features=maxFeatures,n_jobs=40) \
                                                                                                         .fit(dfParent[regressors], np.array(dfParent[response]).ravel())})         
        # label input data using the prediction from the parent RF as this will be unique
        dfBiome['estimate']=np.around(np.array(parentRFDict['RF'].predict(dfBiome[regressors])),decimals=3)

        # populate dictionary of children RFs, each childRF is itself a dictionary similar to the parentRF but now using more than one tree
        # each child is labelled using the prediction value from the parentRF corresponding to its partition
        childrenRFDict = {}
        print('number children:',np.unique(np.around(np.array(parentRFDict['RF'].predict(dfBiome[regressors])),decimals=3)).size)
        for partition in np.unique(np.around(np.array(parentRFDict['RF'].predict(dfBiome[regressors])),decimals=3)):
            dfpartitionBiome = dfBiome.loc[dfBiome['estimate'] == partition]
            childRFDict = {}
            childRFDict.update({'size': dfpartitionBiome[response].shape[0]})
            childRFDict.update({'regressors': regressors})
            childRFDict.update({'regressorsGEE': regressorsGEENames})
            childRFDict.update({'response': response})
            childRFDict.update({'domain':makeDomain(dfpartitionBiome[regressors],domainIndex,domainScaling,domainOffset)})
            childRFDict.update({'RF': RandomForestRegressor(n_estimators=nTrees,min_samples_leaf=minSamplesLeafChild,bootstrap=True,random_state=0,verbose=0,max_depth=maxDepthChild,max_leaf_nodes=maxleafnodesChild,max_features=maxFeatures,n_jobs=40) \
                                     .fit(dfpartitionBiome[regressors], np.array(dfpartitionBiome[response]).ravel())})             
            childrenRFDict.update({partition: childRFDict})

        # assign the childrenRFDict to the parent
        parentRFDict.update({'childrenRFDict':childrenRFDict })      

        #assign the parentRF dict to the calibration data dictionary for trhis biome
        dataDictParent[biome].update({method+response[0]+'parentRFDict':parentRFDict})   
    return dataDictParent

In [39]:
# apply algorithm to data 
def predictClassifier(dataDict,methodDict, method, regressorsNames, response):
    # make generic names for regressors for use in GEE
    regressors = []
    for item in np.arange(1,len(regressorsNames)+1,1):
        regressors.append('x' + str(item))

    # Calibrate hierarchal randforest preeidctors for each biome
    for biome in dataDict.keys():

        print('biome:',biome)

        if ( biome in [1,2,4,6,7]):
            # subset only the regressors 
            dfBiome = dataDict[biome]['DF'][sum([regressorsNames],[])].astype('int')
            dfBiome.columns = regressors

            #apply the parent classifier
            parentRF = methodDict[biome][method+response[0]+'parentRFDict']['RF']
            dfBiome['childNames'] =  np.around(parentRF.predict(dfBiome),decimals=3)
            dataDict[biome]['DF'][method + response[0]+'childNames'] =dfBiome['childNames']

            for partition in np.unique(dfBiome['childNames'] ):
                dfBiome.loc[dfBiome['childNames']==partition,method + response[0]] = methodDict[biome][method+response[0]+'parentRFDict']['childrenRFDict'][partition]['RF'].predict(dfBiome.loc[dfBiome['childNames']==partition][regressors])  
            dataDict[biome]['DF'][method + response[0]] = dfBiome[method + response[0]]
           
    return dataDict

In [23]:
#code the input feature domain by using a linear hash for each row of the input data frame
#the hash algorithm converts each input row into an integer from 0 to 9 by applying the provided scale and offset and then rounding
#is then produces a hash entry for each row by packing the integers consequitively to form a uint64 code
#this implies a limit of at most 18 columns for the input data frame
#returns a list corresonding to hash table of unique coded input rows
def makeDomain(df,domainIndex,domainScaling,domainOffset):
    df = np.array(df) 
    if df.shape[1] < 19 :
        domainIndex = np.array(domainIndex)
        domainScaling = np.array(domainScaling)
        domainOffset = np.array(domainOffset)
    else:
        raise ValueError("More than 18 dimensions in domain")
    return np.uint64(np.unique(np.sum(np.clip(np.around(df* domainScaling + domainOffset,0),0,9) * np.power(10,np.cumsum(domainIndex)-domainIndex[0]),1),0)).tolist()

In [24]:
# parse a sckitlearn decision tree into a R text tree suitable for use in GEE
# for compactness ancillary items like node sample size and residuals are forced to = 1
# this is a blind guess by Richard but seems to work
def make_tree(rf,regressors,decimals=3,maxDepth=20):
    
    # first get the output in sckitlearn text format in a dataframe
    r = export_text(decision_tree=rf,feature_names=regressors,show_weights=True,decimals=decimals,max_depth=maxDepth)
    r = r.splitlines()
    rdf = pd.DataFrame(r,columns = ['rule'])
    
    #identify rules and not leaf values
    isrule = ~rdf['rule'].str.contains('value')
    rulesdf = rdf.loc[isrule]
    
    #determine level in tree and the associated starting based node number
    rdf['level'] = rdf['rule'].str.count(r'(\|)').values.tolist()
    rdf.loc[isrule,'base'] = ((rdf.level).mul(0).add(2)).pow(rdf.level)

    # get the actual tested condition
    rdf.loc[isrule,'condition'] =  rdf.loc[isrule,'rule'].str.extract(r'(x.+)').values.tolist()
    
    # identify leaf nodes and fill in the response value
    rdf.loc[~isrule,'leaf'] = '*'
    rdf['leaf'] = rdf['leaf'].fillna(method='bfill',limit=1)
    rdf.loc[~isrule,'response'] = rdf.loc[~isrule,'rule'].str.extract(r'([+-]?([0-9]*[.])?[0-9]+)')[0].values.tolist()
    rdf['response'] = rdf['response'].fillna(method='bfill')
    
    #discard non rules
    rdf.loc[rdf['leaf'].isna(),'leaf'] = ' '
    rdf = rdf.dropna()

    #dtermine if this is a left or right branch
    rdf['branch'] = rdf['rule'].str.contains(r'(?:\>)').astype('int')
    rdf['node'] = rdf.base + rdf.branch
    rdf.loc[rdf.level==1,'node']=rdf.loc[rdf.level==1,'branch'] + 2
    rdfindex = rdf.index
    
    #asign a node number, this is non trivial and critical for use later
    #read https://www.r-bloggers.com/2022/10/understanding-leaf-node-numbers-when-using-rpart-and-rpart-rules/
    for row in range(2,rdf.shape[0]):
        # find the nearest row above
        df = rdf[0:row]
        if ( (rdf[row:row+1].level.values)[0] > 1 ):
            parentdf = df.loc[df.level == (rdf[row:row+1].level.values-1)[0]].iloc[-1]
            rdf.at[rdfindex[row],'parentbase'] = parentdf.base  
            rdf.at[rdfindex[row],'parentnode'] = parentdf.node  
            rdf.at[rdfindex[row],'node'] = rdf.iloc[row].node + 2 * (  parentdf.node - parentdf.base ) 
    
    # glue together each rule in a big string, add the root node and return as a list
    rdf['phrase'] = rdf.apply(lambda x:  ' ' *(2 * x.level) + str(int(x.node)) + ') ' + x.condition + ' 0 0 ' + str(x.response) + ' ' +x.leaf + '\n', axis=1)
    return ( '1) root 1 1 1 (1)\n'+''.join(rdf['phrase'].values.tolist()))

In [25]:
# This function was taken from the GEEMAP libraries and modified as needed here
def export_trees_to_fc_CCRS(trees,response,regressors,regressorsGEECollectionName,regressorsGEENames,responseGEEScaling,responseGEEOffset,regressorsGEEScaling2, \
                            regressorsGEEScaling,regressorsGEEOffset,domain,domainScaling,domainOffset,asset_id,description="geemap_rf_export"):

    """Function that creates a feature collection with a property tree which contains the string representation of decision trees and exports to ee asset for later use
        together with CCRS tree properties
    args:
        trees (list[str]): list of string representation of the decision trees
        response (str): name of response variable
        regressors (list[str]): list pf strings of names of regressors variables in the created trees
        regressorsGEECollectionName (str) : name of GEE input collection
        regressorsGEENames (list[str]): list of names of the regressors variables in the GEE input collection
      	responseGEEScaling (list[float]): list of scaling values to apply to GEE output image
        responseGEEOffset (list[float]): list of  offset values to apply to GEE output image
        regressorsGEEScaling2 (list[float]): list of scaling values to apply to GEE input collection after initial scale ad offset is applied
        regressorsGEEScaling (list[float]): list of scaling values to apply to GEE input collection
        regressorsGEEOffset (list[float]): list of  offset values to apply to GEE input collection
        domain (list[uint64]): list of domain code values
        domainScaling (list[float]) : list of scaling values to create domain
        domainOffset ( list[float]): list of offset values to create domain
        asset_id (str): ee asset id path to export the feature collection to


    kwargs:
        description (str): optional description to provide export information. default = "geemap_rf_export"

    """
    # create a null geometry point. This is needed to properly export the feature collection
    null_island = ee.Geometry.Point([0, 0])

    # create a list of feature over null island
    # set the tree property as the tree string
    # encode return values (\n) as #, use to parse later

    
    features = [
        ee.Feature(null_island, {"tree": tree.replace("\n", "#"),\
                                 "response": ','.join(response),\
                                 "regressors": ','.join(regressors),\
                                 "regressorsGEECollectionName":regressorsGEECollectionName,\
                                 "regressorsGENames": ','.join(regressorsGEENames),\
                                 "responseGEScaling": ','.join(str(x) for x in responseGEEScaling),\
                                 "responseGEOffset": ','.join(str(x) for x in responseGEEOffset),\
                                 "regressorsGEScaling2": ','.join(str(x) for x in regressorsGEEScaling2),\
                                 "regressorsGEScaling": ','.join(str(x) for x in regressorsGEEScaling),\
                                 "regressorsGEOffset": ','.join(str(x) for x in regressorsGEEOffset),\
                                 # "domain": ','.join(str(x) for x in domain),\
                                 "domainScaling": ','.join(str(x) for x in domainScaling),\
                                 "domainOffset": ','.join(str(x) for x in domainOffset)} ) for tree in trees]
    
    # cast as feature collection
    fc = ee.FeatureCollection(features)

    # get export task and start
    task = ee.batch.Export.table.toAsset(
        collection=fc, description=description, assetId=asset_id
    )
    task.start()

In [26]:
def strings_to_classifier(trees=None,outputMode='REGRESSION'):
    """Function that takes string representation of decision trees and creates a ee.Classifier that can be used with ee objects

    args:
        trees (list[str]): list of string representation of the decision trees
        outputMode [str] : classifier output mode
    returns:
        classifier (ee.Classifier): ee classifier object representing an ensemble decision tree

    """

    # convert strings to ee.String objects
    ee_strings = [ee.String(tree) for tree in trees]

    # pass list of ee.Strings to an ensemble decision tree classifier (i.e. RandomForest)
    classifier = ee.Classifier.decisionTreeEnsemble(ee_strings).setOutputMode(outputMode)

    return classifier


In [27]:
# Get list of all intercomaprison feature collections
def get_asset_list(parent):
    parent_asset = ee.data.getAsset(parent)
    parent_id = parent_asset['name']
    parent_type = parent_asset['type']
    asset_list = []
    child_assets = ee.data.listAssets({'parent': parent_id})['assets']
    for child_asset in child_assets:
        child_id = child_asset['name']
        child_type = child_asset['type']
        if child_type in ['FOLDER','IMAGE_COLLECTION']:
            # Recursively call the function to get child assets
            asset_list.extend(get_asset_list(child_id))
        else:
            asset_list.append(child_id)
    return asset_list

# Read in calibration dictionaries
## There are separate dictionaries for Saturated and UnSaturated samples for each of LAI and FAPAR and for the naive algorithm and the FTL algorithm
## Each dictionary contains one key per biome number 
## Each biome has a dictionary that at minimum includes a key "DF" that corresponds to the calibration data 

In [32]:
# Read in calibration dictionaries found at https://drive.google.com/drive/folders/1fpU0jCLiMVPKqkES8kqRw40jGP38uKmp
# There are separate dictionaries for Saturated and UnSaturated samples for each of LAI and FAPAR and for the naive algorithm and the FTL algorithm
# Each dictionary contains one key per biome number 
# Each biome has a dictionary that at minimum includes a key "DF" that corresponds to the calibration data 
calbiomeDictLAINAIVE= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictLAINAIVE.pkl')
calbiomeDictLAIFTL= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictLAIFTLdf.pkl')

# The dictionarys below will eventually be used but are not currently available

# calbiomeDictLAINAIVESat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictLAINAIVESat.pkl')
# calbiomeDictLAINAIVEUnSat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictLAINAIVEUnSat.pkl')
# calbiomeDictLAIFTLSat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictLAIFTLSat.pkl')
# calbiomeDicLAIFTLUnSat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictLAIFTLUnSat.pkl')

calbiomeDictFAPARNAIVE= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictFAPARNAIVE.pkl')
calbiomeDictFAPARFTL= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictFAPARFTLdf.pkl')

# calbiomeDictFAPARNAIVESat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictFAPARNAIVESat.pkl')
# calbiomeDictFAPARNAIVEUnSat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictFAPARNAIVEUnSat.pkl')
# calbiomeDictFAPARFTLSat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictFAPARFTLSat.pkl')
# calbiomeDictFAPARFTLUnSat= pd.read_pickle('c:/users/rfernand/modisLandsat/code/calbiomeDictFAPARFTLUnSat.pkl')

# Calibrate biome specific Hierarichal RF models and save as a new key in the associated dictionary for each biome

## Calibrate RF for all samples (includes both saturated and unsaturated together) - This is the current implemented solution

In [40]:
# FTL LAI
method = 'FTL'
regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = ['LAI']

# Stuff for GEE
#Specify scaling and index for Domain, currently we only allow one digit per regressor
# Number of digits per regressor
domainIndex = [1,1,1,1,1]

# Scaling per regressor
domainScaling = [ 10/10000, 10/10000,10/10000,10/10000,10/10000]

# Offset per regressor
domainOffset = [ 0,0,0,0,0]

#GEE regressor names
regressorsGEENames =  [ 'SR_B4', 'SR_B5','cosSZA','cosVZA','cosSA'] 

# Calibrate the Hierarchical RF
calbiomeDictLAIFTL= hierarchicalRF(calbiomeDictLAIFTL,calbiomeDictLAINAIVE,regressors,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=10,maxDepthChild=10,minSamplesSplit=2,\
                             maxleafnodesParent=9,  minSamplesLeafParent=1000,maxleafnodesChild= 999, minSamplesLeafChild=1,maxFeatures=4,nTrees = 100)

# Run this if you want to check the predictions
calbiomeDictLAIFTL=  predictClassifier(calbiomeDictLAIFTL,calbiomeDictLAIFTL, 'FTL', regressors, ['LAI'])

biome: 2
biome: 1
biome: 9
biome: 4
biome: 6
biome: 7
biome: 8
biome: 3
biome: 5
biome: 10


In [42]:
# FTL FAPAR
method = 'FTL'
regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = ['FAPAR']

# Stuff for GEE
#Specify scaling and index for Domain, currently we only allow one digit per regressor
# Number of digits per regressor
domainIndex = [1,1,1,1,1]

# Scaling per regressor
domainScaling = [ 10/10000, 10/10000,10/10000,10/10000,10/10000]

# Offset per regressor
domainOffset = [ 0,0,0,0,0]

#GEE regressor names
regressorsGEENames =  [ 'SR_B4', 'SR_B5','cosSZA','cosVZA','cosSA'] 

#Calibrate the Hierarchical RF
calbiomeDictFAPARFTL= hierarchicalRF(calbiomeDictFAPARFTL,calbiomeDictFAPARNAIVE,regressors,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=20,maxDepthChild=20,minSamplesSplit=2,\
                             maxleafnodesParent=9,  minSamplesLeafParent=1000,maxleafnodesChild= 999, minSamplesLeafChild=1,maxFeatures=4,nTrees = 100)

# Run this if you want to check the predictions
# calbiomeDictFAPARFTL=  predictClassifier(calbiomeDictFAPARFTL,calbiomeDictFAPARFTLSat, 'FTLSat', regressors, ['FAPAR'])

biome: 2
Total size  (484803, 6)
Parent size  (11815, 6)
number children: 9
biome: 1
Total size  (2358491, 6)
Parent size  (52030, 6)
number children: 9
biome: 9
Total size  (19139, 6)
Parent size  (3533, 6)
number children: 3
biome: 4
Total size  (1767949, 6)
Parent size  (10323, 6)
number children: 8
biome: 6
Total size  (541660, 6)
Parent size  (4230, 6)
number children: 3
biome: 7
Total size  (376649, 6)
Parent size  (3409, 6)
number children: 3
biome: 8
Total size  (60805, 6)
Parent size  (4747, 6)
number children: 3
biome: 3
Total size  (260189, 6)
Parent size  (5322, 6)
number children: 4
biome: 5
Total size  (215478, 6)
Parent size  (4232, 6)
number children: 3
biome: 10
Total size  (2183, 6)
Parent size  (2183, 6)
number children: 2


## Calibrate Saturated and Unsaturated RFs - Optional

In [ ]:
# FTLSat LAI
method = 'FTLSat'
regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = ['LAI']

# Stuff for GEE
#Specify scaling and index for Domain, currently we only allow one digit per regressor
# Number of digits per regressor
domainIndex = [1,1,1,1,1]

# Scaling per regressor
domainScaling = [ 10/10000, 10/10000,10/10000,10/10000,10/10000]

# Offset per regressor
domainOffset = [ 0,0,0,0,0]

#GEE regressor names
regressorsGEENames =  [ 'SR_B4', 'SR_B5','cosSZA','cosVZA','cosSA'] 

#Calibrate the Hierarchical RF
calbiomeDictLAIFTLSat= hierarchicalRF(calbiomeDictLAIFTLSat,calbiomeDictLAINAIVESat,regressors,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=20,maxDepthChild=20,minSamplesSplit=2,\
                             maxleafnodesParent=999,  minSamplesLeafParent=1000,maxleafnodesChild= 999, minSamplesLeafChild=1,maxFeatures=4,nTrees = 100)

# Run this if you want to check the predictions
# calbiomeDictLAIFTLSat=  predictClassifier(calbiomeDictLAIFTLSat,calbiomeDictLAIFTLSat, 'FTLSat', regressors, ['LAI'])

In [ ]:
# FTLUnSat LAI
method = 'FTLUnSat'
regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = ['LAI']

# Stuff for GEE
#Specify scaling and index for Domain, currently we only allow one digit per regressor
# Number of digits per regressor
domainIndex = [1,1,1,1,1]

# Scaling per regressor
domainScaling = [ 10/10000, 10/10000,10/10000,10/10000,10/10000]

# Offset per regressor
domainOffset = [ 0,0,0,0,0]

#GEE regressor names
regressorsGEENames =  [ 'SR_B4', 'SR_B5','cosSZA','cosVZA','cosSA'] 

#Calibrate the Hierarchical RF
calbiomeDictLAIFTLUnSat= hierarchicalRF(calbiomeDictLAIFTLUnSat,calbiomeDictLAINAIVEUnSat,regressors,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=20,maxDepthChild=20,minSamplesSplit=2,\
                             maxleafnodesParent=999,  minSamplesLeafParent=1000,maxleafnodesChild= 999, minSamplesLeafChild=1,maxFeatures=4,nTrees = 100)

# Run this if you want to check the predictions
# calbiomeDictLAIFTLUnSat=  predictClassifier(calbiomeDictLAIFTLUnSat,calbiomeDictLAIFTLUnSat, 'FTLUnSat', regressors, ['LAI'])

In [ ]:
# FTLSat FAPAR
method = 'FTLSat'
regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = ['FAPAR']

# Stuff for GEE
#Specify scaling and index for Domain, currently we only allow one digit per regressor
# Number of digits per regressor
domainIndex = [1,1,1,1,1]

# Scaling per regressor
domainScaling = [ 10/10000, 10/10000,10/10000,10/10000,10/10000]

# Offset per regressor
domainOffset = [ 0,0,0,0,0]

#GEE regressor names
regressorsGEENames =  [ 'SR_B4', 'SR_B5','cosSZA','cosVZA','cosSA'] 

#Calibrate the Hierarchical RF
calbiomeDictFAPARFTLSat= hierarchicalRF(calbiomeDictFAPARFTLSat,calbiomeDictFAPARNAIVESat,regressors,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=20,maxDepthChild=20,minSamplesSplit=2,\
                             maxleafnodesParent=999,  minSamplesLeafParent=1000,maxleafnodesChild= 999, minSamplesLeafChild=1,maxFeatures=4,nTrees = 100)

# Run this if you want to check the predictions
# calbiomeDictFAPARFTLSat=  predictClassifier(calbiomeDictFAPARFTLSat,calbiomeDictFAPARFTLSat, 'FTLSat', regressors, ['FAPAR'])

In [ ]:
# FTLUnSat FAPAR
method = 'FTLUnSat'
regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = ['FAPAR']

# Stuff for GEE
#Specify scaling and index for Domain, currently we only allow one digit per regressor
# Number of digits per regressor
domainIndex = [1,1,1,1,1]

# Scaling per regressor
domainScaling = [ 10/10000, 10/10000,10/10000,10/10000,10/10000]

# Offset per regressor
domainOffset = [ 0,0,0,0,0]

#GEE regressor names
regressorsGEENames =  [ 'SR_B4', 'SR_B5','cosSZA','cosVZA','cosSA'] 

#Calibrate the Hierarchical RF
calbiomeDictFAPARFTLUnSat= hierarchicalRF(calbiomeDictFAPARUnFTLSat,calbiomeDictFAPARNAIVEUnSat,regressors,regressorsGEENames, response, domainScaling,domainOffset, maxDepthParent=20,maxDepthChild=20,minSamplesSplit=2,\
                             maxleafnodesParent=999,  minSamplesLeafParent=1000,maxleafnodesChild= 999, minSamplesLeafChild=1,maxFeatures=4,nTrees = 100)

# Run this if you want to check the predictions
# calbiomeDictFAPARFTLUnSat=  predictClassifier(calbiomeDictFAPARFTLUnSat,calbiomeDictFAPARFTLUnSat, 'FTLUnSat', regressors, ['FAPAR'])

# Upload trees to GEE

In [13]:
ee.Authenticate()

Enter verification code:  4/1AWtgzh5jD4fCRb5yaX0IicByqAvBxuMVM1HW0y99SxucevGXl6ve7iiTxDs



Successfully saved authorization token.


In [14]:
ee.Initialize()

## Upload algiorithms based on combined saturated and unsaturated samples - this is the current implemented solution

In [ ]:
# upload FTL classiers to gee as feature collections
method = 'FTL'

# response variable name
response = ['LAI']
targetDirectory = 'projects/ee-modis250/assets/FTL_trees_LAI/'
methodDict = calbiomeDictLAIFTL
regressors = ["x1","x2","x3","x4","x5"]

# Scaling and offset required of GEE regressors
regressorsGEECollectionName= "LANDSAT/LC08/C02/T1_L2"
regressorsGEEScaling = [2.75e-05,2.75e-05,1,1,1]
regressorsGEEOffset = [-0.02,-0.02,0,0,0]
regressorsGEEScaling2 = [10000,10000,1,1,1]
responseGEEScaling = [100]
responseGEEOffset = [0]

# parameters for parsing tree
decimals = 3
maxDepth = 10

for biome in [1,2,3,4,5,6,7,8]:
    print('biome', biome)
    parentRFDict = methodDict[biome][method+response[0]+'parentRFDict']
    domain = parentRFDict['domain']
    trees = []
    trees.append(make_tree(parentRFDict['RF'][0],regressors,decimals,maxDepth))
    print('# of children', len(parentRFDict['childrenRFDict']))
    assetID = targetDirectory + method + 'biome' + str(biome) + 'parentRF' 
    export_trees_to_fc_CCRS(trees,response,regressors,regressorsGEECollectionName,regressorsGEENames,responseGEEScaling,responseGEEOffset,regressorsGEEScaling2,regressorsGEEScaling,regressorsGEEOffset,domain,domainScaling,domainOffset,assetID)
    # ee_classifier = strings_to_classifier(trees)
    # print(ee_classifier.getInfo())

    childSize = []
    for partition in parentRFDict['childrenRFDict'].keys():
        print('child #', partition)
        childRFDict = parentRFDict['childrenRFDict'][partition]
        childSize.append(childRFDict['size'])
        trees = []
        for tree in np.squeeze(childRFDict['RF'].estimators_) :
            trees.append(make_tree(tree,regressors,decimals,maxDepth))
        assetID =  targetDirectory + method + 'biome' + str(biome) + 'childRF' + str(int(partition*1000)) 
        domain = [0]
        domainScaling = [0]
        domainOffset = [0]
        export_trees_to_fc_CCRS(trees,response,regressors,regressorsGEECollectionName,regressorsGEENames,responseGEEScaling,responseGEEOffset,regressorsGEEScaling2,regressorsGEEScaling,regressorsGEEOffset,domain,domainScaling,domainOffset,assetID)
        # ee_classifier = strings_to_classifier(trees)
        # print(ee_classifier.getInfo())
    print(childSize)

biome 1
# of children 9
child # 4.363
child # 6.597


In [ ]:
# upload FTL classiers to gee as feature collections
method = 'FTL'

# response variable name
response = ['FAPAR']
targetDirectory = '/FTL_trees_FAPAR/'
methodDict = calbiomeDictLAIFTL

# Scaling and offset required of GEE regressors
regressorsGEECollectionName= "LANDSAT/LC08/C02/T1_L2"
regressorsGEEScaling = [2.75e-05,2.75e-05,1,1,1]
regressorsGEEOffset = [-0.02,-0.02,0,0,0]
regressorsGEEScaling2 = [10000,10000,1,1,1]
responseGEEScaling = [10]
responseGEEOffset = [0]

# parameters for parsing tree
decimals = 3
maxDepth = 20

for biome in [1,2,3,4,5,6,7,8]:
    print('biome', biome)
    parentRFDict = methodDict[biome][method+response[0]+'parentRFDict']
    domain = parentRFDict['domain']
    trees = []
    trees.append(make_tree(parentRFDict['RF'][0],regressors,decimals,maxDepth))
    print('# of children', len(parentRFDict['childrenRFDict']))
    assetID = geemap.ee_user_id() + targetDirectory + method + 'biome' + str(biome) + 'parentRF' 
    export_trees_to_fc_CCRS(trees,response,regressors,regressorsGEECollectionName,regressorsGEENames,responseGEEScaling,responseGEEOffset,regressorsGEEScaling2,regressorsGEEScaling,regressorsGEEOffset,domain,domainScaling,domainOffset,assetID)
    ee_classifier = strings_to_classifier(trees)
    print(ee_classifier.getInfo())

    childSize = []
    for partition in parentRFDict['childrenRFDict'].keys():
        print('child #', partition)
        childRFDict = parentRFDict['childrenRFDict'][partition]
        childSize.append(childRFDict['size'])
        trees = []
        for tree in np.squeeze(childRFDict['RF'].estimators_) :
            trees.append(make_tree(tree,regressors,decimals.maxDepth))
        assetID = geemap.ee_user_id() + targetDirectory + method + 'biome' + str(biome) + 'childRF' + str(int(partition*1000)) 
        domain = [0]
        domainScaling = [0]
        domainOffset = [0]
        mlLocal.export_trees_to_fc_CCRS(trees,response,regressors,regressorsGEECollectionName,regressorsGEENames,regressorsGEEScaling2,regressorsGEEScaling,regressorsGEEOffset,domain,domainScaling,domainOffset,assetID)
        # ee_classifier = strings_to_classifier(trees)
        # print(ee_classifier.getInfo())
    print(childSize)

In [18]:
# # Only run this is you want to delete all GEE assets in a folder
# for asset in get_asset_list('projects/ee-modis250/assets/FTL_trees_LAI'):
#     ee.data.deleteAsset(asset)